# Gaussian State Preparation

Ref [1] outlines how to approximately preparare a 1D Gaussian state

$$

|G(\sigma,\mu)\rangle = \frac{1}{\mathcal{N}} \int dx e^{-\left(\frac{x-\mu}{\sigma}\right)^2} |x\rangle; \quad \mathcal{N}^2 = \sigma \sqrt{2\pi}

$$

which is approximated by a discreted 1DGS (Kitaev-Webb (KW))

$$

|G_{\mathrm{KW}}(\sigma,\mu)\rangle = \sum_{i\in\mathbb{Z}} G_{\mathrm{KW}}(\sigma,\mu;i)|i \rangle

$$

where

$$
G_{\mathrm{KW}}(\sigma,\mu;i) = \frac{1}{f_{\mathrm{KW}(\sigma,\mu)}} e^{\frac{(i-\mu)^2}{4\sigma^2}}, \quad f_{\mathrm{KW}}(\sigma,\mu) = \sum_{i\in\mathbb{Z}} e^{\frac{(i-\mu)^2}{4\sigma^2}}.

$$

In practice, we further approximate 

$$
G_{\mathrm{KW}}(\sigma,\mu,i) \approx \sum_j^{2^m-1}G_{\mathrm{KW}}(\sigma,\mu,i+j2^m).
$$

There are then two approximations: 1) the representation of a Gaussian state defined for continuous values of the position, and 2) the description of this discretized Gaussian using a fixed prescision real numbers, i.e. every real number is associated with a $p$-bit integer. The question is then how to generate this state, and how much do these approximations matter in practice.

From [1], they say the (integer) largest value for a real number is $2^m$, so they use $m$ to be the radix point position (the number of bits to the left of the decimal point).  


In our case we just want to prepare the state

$$

\psi(k) = C e^{-(k-\mu)^2/4\sigma^2}

$$

which can be approximated as


$$

|\psi \rangle = \tilde{C} \sum_n \delta e^{-n^2/(4\tilde{\sigma})^2} |\delta n\rangle

$$

where $k$ is a continuous variable in principle. In practice $k$ lives on a discrete grid given by $[-2\pi n/L, 2\pi/L]$, where $L$ is the size of the supercell, and the range of $n$ is usually fixed by the planewave cutoff, $\frac{1}{2}(k_{\max}-k_{\mathrm{mean}})^2 \le E_{\mathrm{cut}}$, which for us is typically $n \approx 61$, $\delta = 2\pi/L$

[1] suggests two algorithms to generate a Gaussian. The first is a recursive algorithm where the amplitudes $\approx e^{k^2/\sigma^2}$  are prepared on the quantum computer via

$$

|\xi(\sigma, \mu, m)\rangle = |\xi(\sigma/2,\mu/2,m-1)\rangle\otimes|0\rangle + |\xi(\sigma,(\mu-1)/2,m-1\rangle)\otimes|1\rangle, \quad \theta = \arccos\sqrt{\frac{f_{\mathrm{KW}}(\sigma, \mu/2)}{f_{\mathrm{KW}}(\sigma, \mu)}}.

$$

This is problematic because it involves computing $\arccos$ and $f_{\mathrm{KW}}$ on the quantum computer.

The second approach is to use an idea similar to alias sampling described in the linear-T complexity paper. At a high level we can classically compute the amplitudes, load these into a register and probabilistically sample from this distribution such that some uniformly prepared initial state ends up in a correctly weighted state.


[1] https://arxiv.org/pdf/2110.05708.pdf




* Test these classically.
* Cost alias sampling approach (this should be in the PRL).
* Check error given planewave cutoffs we expect and sigma.

## Basic Approximations

Given some Gaussian defined for continuous $x$ in 1D we want to compute the discretization error this introduces (there is no fidelity error if $k$ is identically a RLV).

If we have a Gaussian wavepacket in real space with some width $\sigma_x$

$$

\psi(x) = \frac{1}{\sigma_x\sqrt{2\pi}}e^{-x^2/2\sigma_x^2}

$$

then we have

\begin{align}

\psi(k) &= \frac{1}{\sigma_x\sqrt{2\pi}} \times \sqrt{\pi 2\sigma_x^2}e^{-\pi^2 2 \sigma_r^2 k^2} \\
        &= e^{-k^2/2\sigma_k^2}

\end{align}

where $\sigma_k = \frac{1}{2\pi\sigma_x}$.

If we fix $\sigma_x \approx 1$ bohr, then $\sigma_k \approx \frac{1}{2\pi} \ \mathrm{bohr}^{-1} \approx 0.159 \ \mathrm{bohr}^{-1}$. The momentum space grid spacing from the Deuterium syste is $0.39 \ \mathrm{bohr}^{-1}$

If we want better resolution we can tune the width in real space so that $\sigma_k \le p_k$, where $p_k$ is the  

In [1]:
import numpy as np

def gaussian_real_space(ns, sigma_r, box_length):
    x_scaled = ns / L
    gx = np.exp(-x_scaled**2/(2*sigma_r)**2) / (sigma*np.sqrt(2*np.pi))
    return gx

def gaussian_kspace(ns, sigma_k, box_length):
    g_scaled = ns / L
    gx = np.exp(-g_scaled**2/(2*sigma_k)**2)
    return gx